<a href="https://colab.research.google.com/github/RafaelCaballero/Julio24/blob/main/code/proyecto6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python
Rafa Caballero

## Proyecto 8, regresión logística

Comenzamos cargando algunos datos


In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np

valores = ["AAPL","AMZN","GOOG","NFLX","MSFT","SHEL","LLY"]
for v in valores:
    #print(v)
    df = yf.download(v,
                          start='2008-01-01',
                          end='2024-06-30')

    # además lo renombramos
    df.columns = [c+"_"+v for c in df.columns]
    if v==valores[0]:
        df_total = df
    else:
        df_total = pd.concat([df_total, df], axis=1)

df_total = df_total.pct_change().dropna()


Marcamos con 1 las columnas en las que AAPL-Close supera la media de la serie histórica

In [ ]:
filtro = df_total["Close_AAPL"]> df_total["Close_AAPL"].mean()
df_total.loc[filtro,"label"]=1
df_total.loc[~filtro,"label"]=0
# la ponemos la primera para que se vea mejor
first_column = df_total.pop('label')
df_total.insert(0, 'label', first_column)
df_total

**Ejercicio**

Intentar enteder las siguientes instrucciones ¿qué efecto tienen? (pista: fijarse en la columna "label")

In [ ]:
df = df_total.copy()
df.iloc[0:-1,0] = df.iloc[1:,0]
df = df.iloc[0:-1]
df

**Ejercicio**
Eliminar de df todas las columnas relacionadas con AAPL, porque lo que buscamos es ver si Apple va a tener un incremento a partir del resto. Se puede escribir el nombre de las columnas a borrar a mano

In [ ]:
print(df.columns)
# Solución


**Ejercicio**

Utilizar regresión logística para intentar determinar label a partir del resto de columnas en df. Se puede utilizar la siguiente función para mostrar los resultados:

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
def evaluar(y_test,y_pred):
  k =  cohen_kappa_score(y_test,y_pred)
  print("kappa ",k)
  cm = confusion_matrix(y_test, y_pred, labels=modelo.classes_)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                display_labels=modelo.classes_)
  disp.plot()

  plt.show()

  print(classification_report(y_test, y_pred))
  return k,cm

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer,f1_score, accuracy_score, recall_score,cohen_kappa_score

from sklearn.linear_model import LogisticRegression

### Solución





¿Cuál es el peor valor entre precision y recall que se obtiene? ¿qué indica? ¿a qué puede deberse?

**Ejercicio**

PArtimos de los mismos datos:

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np

valores = ["AAPL","AMZN","GOOG","NFLX","MSFT","SHEL","LLY"]
for v in valores:
    #print(v)
    df = yf.download(v,
                          start='2008-01-01',
                          end='2024-06-30')

    # además lo renombramos
    df.columns = [c+"_"+v for c in df.columns]
    if v==valores[0]:
        df_total = df
    else:
        df_total = pd.concat([df_total, df], axis=1)

df_total = df_total.pct_change().dropna()
df_total

Ahora queremos obtener el incremento de Close_AAPL mediante regresión

En particular se propone:

1) Eliminar todas las columnas relacionadas con APPL excepto CLOSE_APPL

2) Usar regresión lineal para intentar calcularla

Se propone usar los 4 pasos (sin validación cruzada)

In [ ]:
#  Solución


**Ejercicio**

Examinar los coeficientes del modelo ¿Qué indican?






**Ejercicio**

Hacerlo ahora con validación cruzada

**Ejercicio**

Probar algún otro método de regresión como RandomForestRegressor o GradientBoostingRegressor ¿se mejora el error?